In [1]:
from utils.util import *


KeyboardInterrupt



In [ ]:
train_path = '../resource/train_0.8_super_pop.csv'
target_column = "Population code"
input_to_output = [{"input_path": "../fake_genotypes_sequences/new_sequences/discriminator_0.8_test/genotypes.hapt",
                    "output_path": "combine_discriminator_0.8_test.csv"}]

In [ ]:
def init_train_data():
    dataset, _, _, class_to_id = init_dataset(train_path, target_column=target_column, without_extra_data=True)
    train_data = pd.DataFrame(dataset[0])
    train_data[target_column] = pd.DataFrame(np.argmax(dataset[1], axis=1), columns=[target_column])
    train_data["source"] = "Real"
    return train_data, class_to_id

In [ ]:
train_data, class_to_id = init_train_data()

In [ ]:
def concat_with_fake_genotypes(file_path: str, model_results_path: str, train_data):
    tmp_df = pd.read_csv(file_path, sep=' ', header=None)
    pop_column = tmp_df[0].str.replace('Fake_', '')
    tmp_df.drop([0], inplace=True, axis=1)
    tmp_df.columns = [genotype for genotype in range(len(tmp_df.columns))]
    tmp_df[target_column] = pop_column.replace(class_to_id)
    tmp_df["source"] = "Fake"
    tmp_df.reset_index(drop=True)
    concat_df = train_data
    concat_df = concat_df.sample(frac=1, random_state=np.random.RandomState(seed=42))
    concat_df.to_csv(f"../resource/{model_results_path}", index=False)

In [ ]:
for combine_data in input_to_output:
    concat_with_fake_genotypes(combine_data["input_path"], combine_data["output_path"], train_data)

In [ ]:
x = pd.read_csv("resource/combine_discriminator_0.8_test.csv")

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

def plot_pca(df):
    # Extract population code and source columns
    pop_code = df['Population code']
    source = df['source']
    df_without_fake = df[df['source']=='Fake']
    # Drop population code and source columns from the dataframe
    df = df.drop(columns=['Population code', 'source'])
    df_without_fake = df_without_fake.drop(columns=['Population code', 'source'])
    # Perform PCA with 2 components
    pca = PCA(n_components=2)
    PCs = pca.fit_transform(df)
    PCs_df = pd.DataFrame(data=PCs, columns=['PC1', 'PC2'])
    PCs_df['Class'] = source
    real_color = 'blue'
    fake_color = 'red'
    real_idx = PCs_df.index[PCs_df['Class'].str.contains('Real')]
    fake_idx = PCs_df.index[PCs_df['Class'].str.contains('Fake')]
    real_pca = PCs_df.loc[real_idx, ['PC1', 'PC2']]
    fake_pca = PCs_df.loc[fake_idx, ['PC1', 'PC2']]
    plt.scatter(real_pca['PC1'], real_pca['PC2'], c=real_color, label='Real',
                        alpha=0.2)
    plt.scatter(fake_pca['PC1'], fake_pca['PC2'], c=fake_color, label='Fake',
                    alpha=0.2)

    # Add axis labels and a legend
    plt.xlabel('PCA Component 1')
    plt.ylabel('PCA Component 2')
    plt.legend()

    # Display the plot
    plt.show()

In [ ]:
print(pop_column)

In [ ]:
tmp_df = pd.read_csv("../fake_genotypes_sequences/new_sequences/discriminator_0.8_test/genotypes.hapt", sep=' ', header=None)
tmp_df = tmp_df[tmp_df[0] == 'Fake_MXL']
pop_column = tmp_df[0].str.replace('Fake_', '')
tmp_df.drop([0], inplace=True, axis=1)
tmp_df.columns = [genotype for genotype in range(len(tmp_df.columns))]
tmp_df[target_column] = pop_column.replace(class_to_id)
tmp_df["source"] = "Fake"
tmp_df.reset_index(drop=True)
tmp_df.sum()

In [ ]:
%matplotlib inline
import math

generated_genomes_df = pd.read_csv("../fake_genotypes_sequences/new_sequences/discriminator_0.8_test/genotypes.hapt", sep=' ', header=None)
pop_column = generated_genomes_df[0]
generated_genomes_df.drop([0], inplace=True, axis=1)
generated_genomes_df.columns = [genotype for genotype in range(len(generated_genomes_df.columns))]
generated_genomes_df['Type'] = pop_column
# generated_genomes_df.insert(loc=0, column='Type', value=f"Fake_{pop_column}")
plt.rcParams['figure.max_open_warning'] = 50  # set the max number of figures before the warning is triggered to 50

# Make PCA
dataset, _, _, class_to_id = init_dataset(train_path, target_column=target_column, without_extra_data=True)
id_to_class = {i: clas for (clas, i) in class_to_id.items()}
real_sequences = dataset[0]
real_sequences[real_sequences < 0] = 0
real_sequences = np.rint(real_sequences)
real_sequences = pd.DataFrame(real_sequences)
real_sequences['Type'] = pd.DataFrame(np.argmax(dataset[1], axis=1), columns=[target_column])
real_sequences['Type'] = real_sequences['Type'].replace(id_to_class)
real_sequences['Type'] = 'Real_'+real_sequences['Type']
df_all_pca = pd.concat([real_sequences, generated_genomes_df])

pca = PCA(n_components=2)
PCs = pca.fit_transform(df_all_pca.drop(['Type'], axis=1))
PCs_df = pd.DataFrame(data=PCs, columns=['PC1', 'PC2'])
PCs_df['Class'] = list(df_all_pca['Type'])
# Define the colors for real and fake points
real_color = 'blue'
fake_color = 'red'
# Define the populations
populations = list(class_to_id.keys())
# Create a figure with multiple subplots
fig, axs = plt.subplots(nrows=math.ceil((len(populations) + 1) / 2), ncols=2,
                        figsize=(16, len(populations) * 4))
# Loop over each population and plot the real and fake points separately
column_mod = 2
row = 0
alpha_color = 0.3
for i, pop in enumerate(populations):
    # Get the indices of the real and fake points for this population
    real_idx = (PCs_df['Class'] == f'Real_{pop}')
    fake_idx = (PCs_df['Class'] == f'Fake_{pop}')

    # Get the PCA values for the real and fake points
    real_pca = PCs_df.loc[real_idx, ['PC1', 'PC2']]
    fake_pca = PCs_df.loc[fake_idx, ['PC1', 'PC2']]

    # Plot the real and fake points in separate subplots
    axs[row, column_mod % 2].scatter(real_pca['PC1'], real_pca['PC2'], c=real_color, label='Real',
                                     alpha=alpha_color)
    axs[row, column_mod % 2].scatter(fake_pca['PC1'], fake_pca['PC2'], c=fake_color, label='Fake',
                                     alpha=alpha_color)
    axs[row, column_mod % 2].set_xlabel('PC1')
    axs[row, column_mod % 2].set_ylabel('PC2')
    axs[row, column_mod % 2].set_title(f'{pop} - Real vs Fake')
    axs[row, column_mod % 2].legend()
    column_mod += 1
    row += 1 if column_mod % 2 == 0 else 0
# Get the indices of the real and fake points for this population
real_idx = PCs_df.index[PCs_df['Class'].str.contains('Real')]
fake_idx = PCs_df.index[PCs_df['Class'].str.contains('Fake')]
# Get the PCA values for the real and fake points
real_pca = PCs_df.loc[real_idx, ['PC1', 'PC2']]
fake_pca = PCs_df.loc[fake_idx, ['PC1', 'PC2']]
# Plot the real and fake points together in another subplot
axs[row, 1].scatter(real_pca['PC1'], real_pca['PC2'], c=real_color, label='Real',
                    alpha=alpha_color)
axs[row, 1].scatter(fake_pca['PC1'], fake_pca['PC2'], c=fake_color, label='Fake',
                    alpha=alpha_color)
axs[row, 1].set_xlabel('PC1')
axs[row, 1].set_ylabel('PC2')
axs[row, 1].set_title(f'All - Real vs Fake')
axs[row, 1].legend()
plt.tight_layout()
plt.show()

In [ ]:
print(populations)

In [ ]:
df_all_pca['Type'] = df_all_pca['Type'].str.replace('Fake_', '')
df_all_pca['Type'] = df_all_pca['Type'].str.replace('Real_', '')
pop = df_all_pca['Type']
df_all_pca_out = df_all_pca[[i for i in range(len(df_all_pca.columns) - 2)]]
df_all_pca_out = df_all_pca_out.astype(int)
df_all_pca_out['Population code'] = pop
# df_all_pca['Population code'] = df_all_pca['Population code'].replace(class_to_id)
df_all_pca_out.to_csv("../resource/combine_discriminator_0.8_test.csv", index=False)